In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install warnings
!pip install dgl
!pip install texttable

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

2.0.1+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.0/627.0 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.1/504.1 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.3/732.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement warnings (from versions: none)
ERROR: No matching distribution found for warnings
Looking in indexes: https://pypi.org/simple

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
import copy
import torch
from torch_geometric.data import DataLoader


import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ARMAConv

import os
import numpy as np
import os.path as osp
from torch.autograd import grad
from datetime import datetime

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import LEConv, BatchNorm, fps

from datetime import datetime


In [3]:
import numpy as np
import os.path as osp
import pickle
import torch
import torch.utils
import torch.utils.data
import torch.nn.functional as F
from scipy.spatial.distance import cdist
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import InMemoryDataset,Data


In [4]:
import torch
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
from torch_geometric.nn.inits import glorot, zeros
import pdb

class GCNConv(MessagePassing):

    def __init__(self,
                 in_channels,
                 out_channels,
                 improved=False,
                 cached=False,
                 bias=True,
                 edge_norm=True,
                 gfn=False):
        super(GCNConv, self).__init__('add')

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.improved = improved
        self.cached = cached
        self.cached_result = None
        self.edge_norm = edge_norm
        self.gfn = gfn
        self.message_mask = None
        self.weight = Parameter(torch.Tensor(in_channels, out_channels))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        glorot(self.weight)
        zeros(self.bias)
        self.cached_result = None

    @staticmethod
    def norm(edge_index, num_nodes, edge_weight, improved=False, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1), ),
                                     dtype=dtype,
                                     device=edge_index.device)

        edge_weight = edge_weight.view(-1)


        assert edge_weight.size(0) == edge_index.size(1)

        edge_index, edge_weight = remove_self_loops(edge_index, edge_weight)
        edge_index, _ = add_self_loops(edge_index, num_nodes=num_nodes)
        # Add edge_weight for loop edges.
        loop_weight = torch.full((num_nodes, ),
                                 1 if not improved else 2,
                                 dtype=edge_weight.dtype,
                                 device=edge_weight.device)
        edge_weight = torch.cat([edge_weight, loop_weight], dim=0)

        row, col = edge_index
        deg = scatter_add(edge_weight, row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        return edge_index, deg_inv_sqrt[row] * edge_weight * deg_inv_sqrt[col]

    def forward(self, x, edge_index, edge_weight=None):
        """"""

        x = torch.matmul(x, self.weight)
        if self.gfn:
            return x

        if not self.cached or self.cached_result is None:
            if self.edge_norm:
                edge_index, norm = GCNConv.norm(
                    edge_index,
                    x.size(0),
                    edge_weight,
                    self.improved,
                    x.dtype)
            else:
                norm = None
            self.cached_result = edge_index, norm

        edge_index, norm = self.cached_result
        return self.propagate(edge_index, x=x, norm=norm)

    def message(self, x_j, norm):

        if self.edge_norm:
            return norm.view(-1, 1) * x_j
        else:
            return x_j

    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)

In [6]:
from functools import partial
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d, Sequential, ReLU
from torch_geometric.nn import global_mean_pool, global_add_pool, GINConv, GATConv

import random
import pdb
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

layers=3
with_random=True
fc_num=222
hidden=32
eval_random=False
class causalpreCO(torch.nn.Module):
  def __init__(self, num_features,num_classes,gfn=False,collapse=False,residual=False,
                      res_branch="BNConvReLU",
                      global_pool="sum",
                      dropout=0,
                      edge_norm=True):
    super(causalpreCO, self).__init__()
    num_conv_layers = layers
    self.global_pool = global_add_pool

    self.with_random = with_random
    GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)
    self.bnc = BatchNorm1d(hidden)
    self.bno= BatchNorm1d(hidden)
    self.context_convs = GConv(hidden, hidden)
    self.objects_convs = GConv(hidden, hidden)


    hidden_in = num_features
    self.num_classes = num_classes
    hidden_out = num_classes
    self.fc_num = fc_num

    self.object_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )


    # BN initialization.
    for m in self.modules():
        if isinstance(m, (torch.nn.BatchNorm1d)):
            torch.nn.init.constant_(m.weight, 1)
            torch.nn.init.constant_(m.bias,0.0001)

  def forward(self,causal,edge_index,causal_edge_weight,batch,eval_random=True):


    xo = F.relu(self.objects_convs(self.bno(causal), edge_index, causal_edge_weight))

    #xc_logis = self.context_readout_layer(xc)
    #xo_logis = self.objects_readout_layer(xo)
    #xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)

    #return xc_logis, xo_logis, xco_logis
    return self.objects_readout_layer(xo,batch)




  def objects_readout_layer(self,x,batch):
      xo = self.global_pool(x, batch)
      x_logis = F.log_softmax(self.object_mlp(xo), dim=-1)
      return x_logis



In [7]:
class causalpreNO(torch.nn.Module):
  def __init__(self, num_features,num_classes,gfn=False,collapse=False,residual=False,
                      res_branch="BNConvReLU",
                      global_pool="sum",
                      dropout=0,
                      edge_norm=True):
    super(causalpreNO, self).__init__()
    num_conv_layers = layers

    self.global_pool = global_add_pool

    self.with_random = with_random

    GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)

    hidden_in = num_features
    self.num_classes = num_classes
    hidden_out = num_classes
    self.fc_num = fc_num
    self.bnc = BatchNorm1d(hidden)
    self.bno= BatchNorm1d(hidden)
    self.context_convs = GConv(hidden, hidden)
    self.objects_convs = GConv(hidden, hidden)

    self.context_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )

    self.random_readout_mlp=torch.nn.Sequential(
        BatchNorm1d(hidden),
        Linear(hidden, hidden),
        ReLU(),
        BatchNorm1d(hidden),
        Linear(hidden, hidden_out)
    )
    #else:
      #   assert False

    # BN initialization.
    for m in self.modules():
        if isinstance(m, (torch.nn.BatchNorm1d)):
            torch.nn.init.constant_(m.weight, 1)
            torch.nn.init.constant_(m.bias, 0.0001)

  #def forward(self,causal,noncausal,batch,causal_edge_weight,noncausal_edge_weight,edge_index,eval_random=True):
  def forward(self,causal,noncausal,edge_index,causal_edge_weight,noncausal_edge_weight,batch,eval_random=True):

    xc = F.relu(self.context_convs(self.bnc(noncausal), edge_index, noncausal_edge_weight))
    xo = F.relu(self.objects_convs(self.bno(causal), edge_index, causal_edge_weight))

    xc = self.global_pool(xc, batch)
    xo = self.global_pool(xo, batch)

    xc_logis = self.context_readout_layer(xc)
    #xo_logis = self.objects_readout_layer(xo)
    xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)

    return xc_logis, xco_logis


  def context_readout_layer(self, x):


      x_logis = F.log_softmax(self.context_mlp(x), dim=-1)
      return x_logis

  def random_readout_layer(self, xc, xo, eval_random):

      num = xc.shape[0]
      l = [i for i in range(num)]
      if self.with_random:
          if eval_random:
              random.shuffle(l)
      random_idx = torch.tensor(l)
      #if self.args.cat_or_add == "cat":
      #    x = torch.cat((xc[random_idx], xo), dim=1)
      #else:
      x = xc[random_idx] + xo

      x_logis = F.log_softmax(self.random_readout_mlp(x), dim=-1)
      return x_logis


In [8]:

class CausalGAN(torch.nn.Module):
    """GCN with BN and residual connection."""
    def __init__(self, num_features,
                       num_classes,
                       gfn=False,
                       collapse=False,
                       residual=False,
                       res_branch="BNConvReLU",
                       global_pool="sum",
                       dropout=0.2,
                       edge_norm=True):
        super(CausalGAN, self).__init__()
        num_conv_layers = layers
        #hidden = args.hidden
        #self.args = args
        hidden=32
        head=4
        self.global_pool = global_add_pool
        self.dropout = dropout
        self.with_random = with_random
        self.without_node_attention = False
        self.without_edge_attention = False
        GConv = partial(GCNConv, edge_norm=edge_norm, gfn=gfn)

        hidden_in = num_features
        self.num_classes = num_classes
        hidden_out = num_classes
        self.fc_num = fc_num
        self.bn_feat = BatchNorm1d(hidden_in)
        self.conv_feat = GCNConv(hidden_in, hidden, gfn=True) # linear transform
        self.bns_conv = torch.nn.ModuleList()
        self.convs = torch.nn.ModuleList()

        for i in range(num_conv_layers):
            self.bns_conv.append(BatchNorm1d(hidden))
            self.convs.append(GATConv(hidden, int(hidden / head), heads=head, dropout=dropout))

        self.edge_att_mlp = nn.Linear(hidden * 2, 2)
        self.node_att_mlp = nn.Linear(hidden, 2)
        self.bnc = BatchNorm1d(hidden)
        self.bno= BatchNorm1d(hidden)
        self.context_convs = GConv(hidden, hidden)
        self.objects_convs = GConv(hidden, hidden)
        self.relu = nn.ReLU(inplace=False)

        for m in self.modules():
            if isinstance(m, (torch.nn.BatchNorm1d)):
                torch.nn.init.constant_(m.weight, 1)
                torch.nn.init.constant_(m.bias, 0.0001)

    def forward(self, data, eval_random=True):

        x = data.x if data.x is not None else data.feat
        edge_index,edge_attr,batch = data.edge_index,data.edge_attr,data.batch
        row, col = edge_index
        x = self.bn_feat(x)
        x = self.relu(self.conv_feat(x, edge_index,edge_weight=edge_attr))

        for i, conv in enumerate(self.convs):
            x = self.bns_conv[i](x)
            x = self.relu(conv(x, edge_index))

        edge_rep = torch.cat([x[row], x[col]], dim=-1)

        if self.without_edge_attention:
            edge_att = 0.5 * torch.ones(edge_rep.shape[0], 2).to(device)
        else:
            edge_att = F.softmax(self.edge_att_mlp(edge_rep), dim=-1)
        edge_weight_c = edge_att[:, 0]
        edge_weight_o = edge_att[:, 1]

        if self.without_node_attention:
            node_att = 0.5 * torch.ones(x.shape[0], 2).to(device)
        else:
            node_att = F.softmax(self.node_att_mlp(x), dim=-1)
        xc = node_att[:, 0].view(-1, 1) * x
        xo = node_att[:, 1].view(-1, 1) * x

        #print(edge_weight_o)
        #xc = F.relu(self.context_convs(self.bnc(xc), edge_index, edge_weight_c))
        #xo = F.relu(self.objects_convs(self.bno(xo), edge_index, edge_weight_o))
        #node_of_graph=xo.shape[0]
        #causal_part,causal_node=split_graph(graph_x=xo,node_of_graph=node_of_graph)
        #noncausal_part,noncausal_node=split_graph(graph_x=xc,node_of_graph=node_of_graph,type_of_graph=False)

        #xc = self.global_pool(xc, batch)
        #xo = self.global_pool(xo, batch)

        #xc_logis = self.context_readout_layer(xc)
        #xo_logis = self.objects_readout_layer(xo)
        #xco_logis = self.random_readout_layer(xc, xo, eval_random=eval_random)

      # return (xo,edge_index,edge_weight_o),(xc,edge_index,edge_weight_c),batch
        return (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch
        #return (causal_attention,causal_part,causal_node),(noncausal_attention,noncausal_part,noncausal_node),batch
        #return (xo,causal_part,causal_node),(xc,noncausal_part,noncausal_node),batch




In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from drive.MyDrive.gsst2.graphsst2_dataset import get_dataset, get_dataloader
import networkx as nx
from torch_geometric.utils import to_networkx
def generate_node_feature(data):
  G = to_networkx(data, to_undirected=True) # Convert to networkx graph

  # Calculate eigenvector centrality
  centrality = nx.eigenvector_centrality_numpy(G)

  # Convert the centrality dictionary to a list maintaining the order of nodes
  node_features = [centrality[node] for node in G.nodes]

  # Convert to tensor
  node_features=torch.tensor(node_features, dtype=torch.float).view(-1, 1)

  return node_features

In [10]:
from torch_geometric.datasets import TUDataset
import os.path as osp
from torch.utils.data import random_split
from torch_geometric.datasets import TUDataset

path ='/content/drive/MyDrive/'
# Load the COLLAB dataset
dataset = TUDataset(root='/content/drive/MyDrive/', name='COLLAB')

# Print some information about the dataset
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the first graph in the dataset
data = dataset[1]
print('\nFirst graph in the dataset:')
print('======================')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {"Yes" if data.contains_isolated_nodes() else "No"}')
print(f'Contains self-loops: {"Yes" if data.contains_self_loops() else "No"}')
print(f'Is undirected: {"Yes" if data.is_undirected() else "No"}')

train_size = int(len(dataset) * 0.7)
val_size = int(len(dataset) * 0.15)
test_size = len(dataset) - train_size - val_size
print(f'training size:{train_size}')
print(f'valating size:{val_size}')
print(f'testing size:{test_size}')

# Generate the splits
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
training=[]
valating=[]
testing=[]
for data in train_dataset:
  data.x=generate_node_feature(data)
  training.append(data)
for data in val_dataset:
  data.x=generate_node_feature(data)
  valating.append(data)
for data in test_dataset:
  data.x=generate_node_feature(data)
  testing.append(data)
print(f'training size:{len(training)}')
print(f'valating size:{len(valating)}')
print(f'testing size:{len(testing)}')
train_loader = DataLoader(training, batch_size=32, shuffle=True)
val_loader = DataLoader(valating, batch_size=32, shuffle=False)
test_loader = DataLoader(testing, batch_size=1, shuffle=False)

Dataset: COLLAB(5000):
Number of graphs: 5000
Number of features: 0
Number of classes: 3

First graph in the dataset:
Number of nodes: 52
Number of edges: 2188
Average node degree: 42.08
Contains isolated nodes: No
Contains self-loops: No
Is undirected: Yes
training size:3500
valating size:750
testing size:750


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'contains_isolated_nodes' is deprecated, use 'has_isolated_nodes' instead
  warnings.warn(out)
/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'contains_self_loops' is deprecated, use 'has_self_loops' instead
  warnings.warn(out)


training size:3500
valating size:750
testing size:750


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [11]:
feat_dim=1
num_classes = 3
causal_model2 = CausalGAN(feat_dim,num_classes).to(device)
predictno_model2=causalpreNO(feat_dim,num_classes).to(device)
predictco_model2=causalpreCO(feat_dim,num_classes).to(device)
model_optimizer2 = torch.optim.Adam(
            list(causal_model2.parameters()) +
            list(predictno_model2.parameters())+list(predictco_model2.parameters()),
            lr=0.001)
Epo=500
lr_scheduler = CosineAnnealingLR(model_optimizer2, T_max=Epo, eta_min=1e-6, last_epoch=-1, verbose=False)

In [13]:
import time
import json

loss_value=[]
loss_value_valation=[]
c=0.5
o=1
co=0.5
def num_graphs(data):
  if data.batch is not None:
      return data.num_graphs
  else:
      return data.x.size(0)
from tqdm import tqdm
Epo=500
for epoch in range(Epo):
  #model.train()
  causal_model2.train()
  predictno_model2.train()
  predictco_model2.train()
  total_loss = 0
  total_loss_c = 0
  total_loss_o = 0
  total_loss_co = 0
  correct_o = 0
  nb=0
  print(f"-----training-------{epoch}")
  loop = tqdm(enumerate(train_loader),total=len(train_loader))
  for it, data in loop:
#  for it, data in enumerate(train_loader):
      nb+=1
      model_optimizer2.zero_grad()
      data = data.to(device)

      one_hot_target = data.y.view(-1)
      #causal,noncausal,batch=causal_model(data)
      (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)
      #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
      c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
      uniform_target = torch.ones_like(c_logs, dtype=torch.float).to(device)/num_classes
     # sim=F.cosine_similarity(causal_attention,noncausal_attention).mean()
      #print(o_logs)
      #print(type(o_logs))
      c_loss = F.kl_div(c_logs, uniform_target,reduction='batchmean')
      #print(f'causal loss{c_loss}')
      o_loss = F.nll_loss(o_logs,one_hot_target)
      #print(f'noncausal loss{o_loss}')
      co_loss = F.nll_loss(co_logs,one_hot_target)
      #print(f'com causal loss{co_loss}')
      loss = c * c_loss + o * o_loss + co * co_loss

      pred_o = o_logs.max(1)[1]
      correct_o += pred_o.eq(data.y.view(-1)).sum().item()
      loss.backward()
      total_loss += loss.item() #* num_graphs(data)
      total_loss_c += c_loss.item() #* num_graphs(data)
      total_loss_o += o_loss.item() #* num_graphs(data)
      total_loss_co += co_loss.item()# * num_graphs(data)
      model_optimizer2.step()
      loop.set_description(f"Epoch [{epoch}/{Epo}]")
      loop.set_postfix(loss = loss.item())

  #num = len(train_loader.dataset)
  total_loss = total_loss / nb
  total_loss_c = total_loss_c / nb
  print(f'number of {epoch} with total loss:{total_loss}')
  loss_value.append(total_loss)
  total_loss_o = total_loss_o / nb
  total_loss_co = total_loss_co / nb
  correct_o = correct_o / nb
  lr_scheduler.step()
  with torch.no_grad():
    correct=0
    correct_c=0
    correct_o=0
    print(f"------valation---------{epoch}")
    causal_model2.eval()
    predictno_model2.eval()
    predictco_model2.eval()
    for data in val_loader:
      (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)
      #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
      c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
      o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
      pred = co_logs.max(1)[1]
      pred_c = c_logs.max(1)[1]
      pred_o = o_logs.max(1)[1]
      correct += pred.eq(data.y.view(-1)).sum().item()
      correct_c += pred_c.eq(data.y.view(-1)).sum().item()
      correct_o += pred_o.eq(data.y.view(-1)).sum().item()
    acc_co = correct / len(val_loader.dataset)
    acc_c = correct_c / len(val_loader.dataset)
    acc_o = correct_o / len(val_loader.dataset)
    print(f"causal val accuracy:{acc_co}")
    loss_value_valation.append(acc_co)
    dictionary={"number of epoch":epoch,
                "training loss list":loss_value,
                "valation accuracy list":loss_value_valation}

    # Serializing json
    json_object = json.dumps(dictionary,indent=3)

    # Writing to sample.json
   # with open("/content/drive/MyDrive/running_RQ2/COLLAB/CAL/CAL_tl_va_e.json", "w") as outfile:
   #     outfile.write(json_object)

    #torch.save(causal_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430cau_my6000.pt')
    #torch.save(predictco_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430caupred_my6000.pt')
    #torch.save(predictno_model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430noncaupred_my6000.pt')
    #torch.save(model_heter.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/430heter6000.pt')
   # torch.save({
   #         'causal_model.state_dic': causal_model2.state_dict(),
   #         'predictco_model_state_dict()': predictco_model2.state_dict(),
   #         'predictno_model_state_dict()': predictno_model2.state_dict(),
   #         'opt':model_optimizer2.state_dict()
   #         }, '/content/drive/MyDrive/running_RQ2/COLLAB/CAL/NLP_allmodel_cal.pt')
    if(epoch>20):
      check=abs(acc_o-loss_value_valation[len(loss_value_valation)-20])/20
      if(check<=0.01):
        break


  #model_optimizer.zero_grad()
  #total_loss.backward()
  #model_optimizer.step()





-----training-------0


Epoch [0/500]: 100%|██████████| 110/110 [01:44<00:00,  1.05it/s, loss=1.21]


number of 0 with total loss:1.4154795375737277
------valation---------0
causal val accuracy:0.6533333333333333
-----training-------1


Epoch [1/500]: 100%|██████████| 110/110 [01:44<00:00,  1.05it/s, loss=1.24]


number of 1 with total loss:1.266071183573116
------valation---------1
causal val accuracy:0.6466666666666666
-----training-------2


...

number of 19 with total loss:0.9542289186607708
------valation---------19
causal val accuracy:0.716
-----training-------20


Epoch [20/500]: 100%|██████████| 110/110 [01:49<00:00,  1.00it/s, loss=1.58]


number of 20 with total loss:0.9575496359304948
------valation---------20
causal val accuracy:0.716
-----training-------21


Epoch [21/500]: 100%|██████████| 110/110 [01:49<00:00,  1.00it/s, loss=0.862]


number of 21 with total loss:0.9545520359819586
------valation---------21
causal val accuracy:0.7013333333333334


In [ ]:
correct=0
correct_c=0
correct_o=0
print(f"------test---------{0000000}")
causal_model2.eval()
predictno_model2.eval()
predictco_model2.eval()
for data in test_loader:
  (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)
  #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
  c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
  o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
  pred = co_logs.max(1)[1]
  pred_c = c_logs.max(1)[1]
  pred_o = o_logs.max(1)[1]
  correct += pred.eq(data.y.view(-1)).sum().item()
  correct_c += pred_c.eq(data.y.view(-1)).sum().item()
  correct_o += pred_o.eq(data.y.view(-1)).sum().item()
acc_co = correct / len(test_loader.dataset)
acc_c = correct_c / len(test_loader.dataset)
acc_o = correct_o / len(test_loader.dataset)
print(f"com causal test accuracy:{acc_co}")
print(f"causal test accuracy:{acc_o}")
dictionary={"number of epoch":epoch,
            "training loss list":loss_value,
            "valation accuracy list":loss_value_valation,
            "test accuracy value":acc_co}

# Serializing json
json_object = json.dumps(dictionary,indent=4)

# Writing to sample.json
with open("/content/drive/MyDrive/running_RQ2/COLLAB/CAL/CAL_tl_va_e.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
'''test_loader = DataLoader(testing_final, batch_size=1, shuffle=False)

correct=0
correct_c=0
correct_o=0
print(f"------test---------{0000000}")
causal_model2.eval()
predictno_model2.eval()
predictco_model2.eval()
for data in test_loader:
  (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)
  #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)
  c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)
  o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)
  pred = co_logs.max(1)[1]
  pred_c = c_logs.max(1)[1]
  pred_o = o_logs.max(1)[1]
  correct += pred.eq(data.y.view(-1)).sum().item()
  correct_c += pred_c.eq(data.y.view(-1)).sum().item()
  correct_o += pred_o.eq(data.y.view(-1)).sum().item()
acc_co = correct / len(test_loader.dataset)
acc_c = correct_c / len(test_loader.dataset)
acc_o = correct_o / len(test_loader.dataset)
print(f"com causal test accuracy:{acc_co}")
print(f"causal test accuracy:{acc_o}")
#dictionary={"number of epoch":epoch,
#            "training loss list":loss_value,
#            "valation accuracy list":loss_value_valation,
#            "test accuracy value":acc_co}'''

# Serializing json
#json_object = json.dumps(dictionary,indent=4)

# Writing to sample.json
#with open("/content/drive/MyDrive/running_cal_mnist/number_tl_va_e095.json", "w") as outfile:
#    outfile.write(json_object)'''


'test_loader = DataLoader(testing_final, batch_size=1, shuffle=False)\n\ncorrect=0\ncorrect_c=0\ncorrect_o=0\nprint(f"------test---------{0000000}")\ncausal_model2.eval()\npredictno_model2.eval()\npredictco_model2.eval()\nfor data in test_loader:\n  (xo,edge_weight_o),(xc,edge_weight_c),edge_index,batch=causal_model2(data)\n  #c_logs, o_logs, co_logs = predict_model(causal=causal,noncausal=noncausal,batch=batch,causal_edge_weight=edge_weight_causal,noncausal_edge_weight=edge_weight_noncausal,edge_index=edge_index)\n  c_logs,co_logs = predictno_model2(causal=xo,noncausal=xc,edge_index=edge_index,causal_edge_weight=edge_weight_o,noncausal_edge_weight=edge_weight_c,batch=batch)\n  o_logs=predictco_model2(causal=xo,edge_index=edge_index,causal_edge_weight=edge_weight_o,batch=batch)\n  pred = co_logs.max(1)[1]\n  pred_c = c_logs.max(1)[1] \n  pred_o = o_logs.max(1)[1]\n  correct += pred.eq(data.y.view(-1)).sum().item()\n  correct_c += pred_c.eq(data.y.view(-1)).sum().item()\n  correct_o += 